## Train model identify if two images are the same turtle

### Import libraries

In [1]:
import os
import glob
import json
from os.path import join

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import random

from ShellRec.data_utils.prepare_photos import get_img_graph, split_graph
from ShellRec.model import TurtleDiff 
from ShellRec.dataset import TurtlePair 
from ShellRec.train import train

In [2]:
random.seed(42)

### Preprocessing photo files

In [3]:
all_train = get_img_graph(path = "../dataset/BoxTurtle", drop_p=[0.99,0])
split_graph(all_train, save_path = "../dataset")
_ = get_img_graph(path = "../dataset/BoxTurtle_holdout", 
                  file_to_save = "../dataset/BoxTurtle_holdout.json")

### Prepare torch dataset

In [7]:
## image transformations
transform_train = timm.data.create_transform(384, is_training = True, 
                                   auto_augment = "rand-m9-mstd0.5")


transform_test = transforms.Compose([
        transforms.Resize((384,384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[.5,.5,.5], 
                             std=[0.5, 0.5, 0.5])
])

train_set = TurtlePair(data_file='../dataset/train.json',transform=transform_train)
val_set = TurtlePair(data_file='../dataset/val.json',transform=transform_test)

# Set up datasets and dataloaders
train_loader = DataLoader( TurtlePair(data_file='../dataset/train.json', 
                                             transform=transform_train), 
                                             batch_size = 64)
val_loader = DataLoader( TurtlePair(data_file='../dataset/val.json', 
                                           transform=transform_test), 
                                           batch_size = 64)

### training

Get the model set up, use a pretrained model as backbone, and add a new head to it.

In [8]:
torch.hub.set_dir('../pretrained/')

In [5]:
#device = "cpu" #
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   

model = TurtleDiff('vit_base_patch16_384') # use a vit backbone
#model = TurtleDiff('resnet50') # use a resnet backbone
optimizer = optim.Adam(model.parameters(), lr=2e-3)
criterion = nn.CrossEntropyLoss()
num_epochs = 20

Training loop

In [9]:
loss_list = train(model, optimizer, criterion, train_loader, val_loader, device, num_epochs,
      save_path = "../pretrained")

Epoch: 1/20


100%|██████████| 19/19 [09:54<00:00, 31.31s/it]


Epoch: 1/20, Training Loss: 0.7473


100%|██████████| 3/3 [01:12<00:00, 24.30s/it]


Epoch: 1/20, Validation Accuracy: 42.76%
Epoch: 2/20


...

Epoch: 19/20, Validation Accuracy: 72.37%
Epoch: 20/20


100%|██████████| 19/19 [11:14<00:00, 35.51s/it]


Epoch: 20/20, Training Loss: 0.4443


100%|██████████| 3/3 [01:11<00:00, 23.77s/it]

Epoch: 20/20, Validation Accuracy: 76.97%
